In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [11]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_donghua_duanpian=pd.DataFrame(columns=columns)
channel_donghua_duanpian

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[1]/div/a/span').click()  #选择“动画”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[1]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'动画'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').click()  #选择“短片·手书·配音”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').text
sub_channel

'短片·手书·配音'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [12]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_donghua_duanpian.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 08:54:22.251704


<ipython-input-12-b07483d818d6>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-b07483d818d6>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-b07483d818d6>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1wJ411H71u视频属性的抓取
BV号为1wJ411H71u的视频属性抓取结束


<ipython-input-12-b07483d818d6>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1MJ411C7ie视频属性的抓取
BV号为1MJ411C7ie的视频属性抓取结束
开始BV号为1RJ411j7kr视频属性的抓取
BV号为1RJ411j7kr的视频属性抓取结束
开始BV号为1qJ411H76x视频属性的抓取
BV号为1qJ411H76x的视频属性抓取结束
开始BV号为13J411j7fr视频属性的抓取
BV号为13J411j7fr的视频属性抓取结束
开始BV号为1fJ411771a视频属性的抓取
BV号为1fJ411771a的视频属性抓取结束
开始BV号为1H7411Y7dw视频属性的抓取
BV号为1H7411Y7dw的视频属性抓取结束
开始BV号为1R7411q7BS视频属性的抓取
BV号为1R7411q7BS的视频属性抓取结束
开始BV号为1KJ411C7oA视频属性的抓取
BV号为1KJ411C7oA的视频属性抓取结束
开始BV号为1KJ411C7m6视频属性的抓取
BV号为1KJ411C7m6的视频属性抓取结束
开始BV号为1o7411B7G5视频属性的抓取
BV号为1o7411B7G5的视频属性抓取结束
开始BV号为1GJ41177fk视频属性的抓取
BV号为1GJ41177fk的视频属性抓取结束
开始BV号为1G7411B7Jy视频属性的抓取
BV号为1G7411B7Jy的视频属性抓取结束
开始BV号为1u7411r7EA视频属性的抓取
BV号为1u7411r7EA的视频属性抓取结束
开始BV号为1DJ41157Bn视频属性的抓取
BV号为1DJ41157Bn的视频属性抓取结束
开始BV号为13J411E7st视频属性的抓取
BV号为13J411E7st的视频属性抓取结束
开始BV号为1Q7411z7Da视频属性的抓取
BV号为1Q7411z7Da的视频属性抓取结束
开始BV号为177411r7po视频属性的抓取
BV号为177411r7po的视频属性抓取结束
开始BV号为1Z7411e7pz视频属性的抓取
BV号为1Z7411e7pz的视频属性抓取结束
开始BV号为1TJ411H7h8视频属性的抓取
BV号为1TJ411H7h8的视频属性抓取结束
开始BV号为1V7411z7Yr视频属性的抓取
BV号为1V7411z7Yr的视频属性抓取结束
开始BV号为1m7411z7DX视频属性的抓取
BV号为1m7411z7DX的视

...

BV号为17E411A7ZL的视频属性抓取结束
开始BV号为1tE411s7zP视频属性的抓取
BV号为1tE411s7zP的视频属性抓取结束
开始BV号为1gE41137J7视频属性的抓取
BV号为1gE41137J7的视频属性抓取结束
开始BV号为167411d7fg视频属性的抓取
BV号为167411d7fg的视频属性抓取结束
开始BV号为1YE411w7AF视频属性的抓取
BV号为1YE411w7AF的视频属性抓取结束
开始BV号为1HE411x74m视频属性的抓取
BV号为1HE411x74m的视频属性抓取结束
开始BV号为1sE411W7qU视频属性的抓取
BV号为1sE411W7qU的视频属性抓取结束
开始BV号为1ZE411P7s2视频属性的抓取
BV号为1ZE411P7s2的视频属性抓取结束
开始BV号为1vK4y1C7GA视频属性的抓取
BV号为1vK4y1C7GA的视频属性抓取结束
开始BV号为1j7411m75Z视频属性的抓取
BV号为1j7411m75Z的视频属性抓取结束
开始BV号为1xE411j7Ch视频属性的抓取
BV号为1xE411j7Ch的视频属性抓取结束
开始BV号为1y7411m7k6视频属性的抓取
BV号为1y7411m7k6的视频属性抓取结束
开始BV号为1UE411j7e1视频属性的抓取
BV号为1UE411j7e1的视频属性抓取结束
开始BV号为1HE411u7tj视频属性的抓取
BV号为1HE411u7tj的视频属性抓取结束
开始BV号为1qE411T7D5视频属性的抓取
BV号为1qE411T7D5的视频属性抓取结束
开始BV号为1fE411L7vV视频属性的抓取
BV号为1fE411L7vV的视频属性抓取结束
开始BV号为1RE41137mt视频属性的抓取
BV号为1RE41137mt的视频属性抓取结束
开始BV号为1aA41187P4视频属性的抓取
BV号为1aA41187P4的视频属性抓取结束
开始BV号为1aE41177SP视频属性的抓取
开始BV号为1aE41177SP视频属性的抓取
BV号为1aE41177SP的视频属性抓取结束
开始BV号为1UE411L72G视频属性的抓取
BV号为1UE411L72G的视频属性抓取结束
开始BV号为1ME411j7Ea视频属性的抓取
BV号为1ME411j7Ea的视

BV号为1nQ4y1T7sT的视频属性抓取结束
开始BV号为1cz411b7M1视频属性的抓取
BV号为1cz411b7M1的视频属性抓取结束
开始BV号为1Bg4y187s3视频属性的抓取
BV号为1Bg4y187s3的视频属性抓取结束
开始BV号为1QV411R7EZ视频属性的抓取
BV号为1QV411R7EZ的视频属性抓取结束
开始BV号为1Uz411B7MK视频属性的抓取
BV号为1Uz411B7MK的视频属性抓取结束
开始BV号为1U54y1Q7mL视频属性的抓取
BV号为1U54y1Q7mL的视频属性抓取结束
开始BV号为1EQ4y1T7Zn视频属性的抓取
BV号为1EQ4y1T7Zn的视频属性抓取结束
开始BV号为1gQ4y1K7dt视频属性的抓取
BV号为1gQ4y1K7dt的视频属性抓取结束
开始BV号为1o54y1975V视频属性的抓取
BV号为1o54y1975V的视频属性抓取结束
开始BV号为1H64y1M7Nd视频属性的抓取
BV号为1H64y1M7Nd的视频属性抓取结束
开始BV号为1KA411q7nx视频属性的抓取
BV号为1KA411q7nx的视频属性抓取结束
开始BV号为11V411d7jG视频属性的抓取
BV号为11V411d7jG的视频属性抓取结束
开始BV号为11A411t7Mn视频属性的抓取
BV号为11A411t7Mn的视频属性抓取结束
开始BV号为19k4y1678w视频属性的抓取
BV号为19k4y1678w的视频属性抓取结束
开始BV号为19z411v7bd视频属性的抓取
BV号为19z411v7bd的视频属性抓取结束
开始BV号为1Ck4y167en视频属性的抓取
BV号为1Ck4y167en的视频属性抓取结束
开始BV号为12a4y1e73b视频属性的抓取
BV号为12a4y1e73b的视频属性抓取结束
开始BV号为1nz411z7JA视频属性的抓取
BV号为1nz411z7JA的视频属性抓取结束
开始BV号为185411s7yc视频属性的抓取
BV号为185411s7yc的视频属性抓取结束
开始BV号为12C4y1W7cx视频属性的抓取
BV号为12C4y1W7cx的视频属性抓取结束
开始BV号为1Dt4y117Qc视频属性的抓取
BV号为1Dt4y117Qc的视频属性抓取结束
开始BV号为1Np4y1X7mr

BV号为1Ea4y1v7Bo的视频属性抓取结束
开始BV号为1q54y1D7eF视频属性的抓取
BV号为1q54y1D7eF的视频属性抓取结束
开始BV号为1Rv411q7mn视频属性的抓取
BV号为1Rv411q7mn的视频属性抓取结束
开始BV号为1Bz4y1Q7iv视频属性的抓取
BV号为1Bz4y1Q7iv的视频属性抓取结束
开始BV号为19i4y1s7p9视频属性的抓取
BV号为19i4y1s7p9的视频属性抓取结束
开始BV号为1Zi4y1G7KH视频属性的抓取
BV号为1Zi4y1G7KH的视频属性抓取结束
开始BV号为17t4y197pZ视频属性的抓取
BV号为17t4y197pZ的视频属性抓取结束
开始BV号为1GC4y1a7Nt视频属性的抓取
BV号为1GC4y1a7Nt的视频属性抓取结束
开始BV号为1xt4y1975D视频属性的抓取
BV号为1xt4y1975D的视频属性抓取结束
开始BV号为1Zt4y1C7GC视频属性的抓取
BV号为1Zt4y1C7GC的视频属性抓取结束
开始BV号为15k4y1z7hc视频属性的抓取
BV号为15k4y1z7hc的视频属性抓取结束
开始BV号为1aa4y1v7HT视频属性的抓取
BV号为1aa4y1v7HT的视频属性抓取结束
开始BV号为19T4y1E7wY视频属性的抓取
开始BV号为19T4y1E7wY视频属性的抓取
BV号为19T4y1E7wY的视频属性抓取结束
开始BV号为1N54y1Q7kY视频属性的抓取
BV号为1N54y1Q7kY的视频属性抓取结束
开始BV号为1D5411W7pY视频属性的抓取
BV号为1D5411W7pY的视频属性抓取结束
开始BV号为1ti4y1x7Yn视频属性的抓取
BV号为1ti4y1x7Yn的视频属性抓取结束
开始BV号为18C4y1p7sN视频属性的抓取
BV号为18C4y1p7sN的视频属性抓取结束
开始BV号为1Ck4y1z71Q视频属性的抓取
BV号为1Ck4y1z71Q的视频属性抓取结束
开始BV号为1Zz4y1Q7CP视频属性的抓取
BV号为1Zz4y1Q7CP的视频属性抓取结束
开始BV号为1sV411r7th视频属性的抓取
BV号为1sV411r7th的视频属性抓取结束
开始BV号为18K4y147MG视频属性的抓取
开始BV号为18K4y147MG

开始BV号为1ZD4y1U7Kh视频属性的抓取
BV号为1ZD4y1U7Kh的视频属性抓取结束
开始BV号为19V411z7wk视频属性的抓取
BV号为19V411z7wk的视频属性抓取结束
开始BV号为1sf4y197sM视频属性的抓取
BV号为1sf4y197sM的视频属性抓取结束
开始BV号为1Ap4y1q7WE视频属性的抓取
BV号为1Ap4y1q7WE的视频属性抓取结束
开始BV号为18541187AP视频属性的抓取
BV号为18541187AP的视频属性抓取结束
开始BV号为1KV411S73h视频属性的抓取
BV号为1KV411S73h的视频属性抓取结束
开始BV号为1u5411h7H2视频属性的抓取
BV号为1u5411h7H2的视频属性抓取结束
开始BV号为1Cv411y7tc视频属性的抓取
BV号为1Cv411y7tc的视频属性抓取结束
开始BV号为1m54y1e76k视频属性的抓取
BV号为1m54y1e76k的视频属性抓取结束
开始BV号为1zV411z7Dv视频属性的抓取
BV号为1zV411z7Dv的视频属性抓取结束
开始BV号为14V411U7zu视频属性的抓取
BV号为14V411U7zu的视频属性抓取结束
开始BV号为1Wv411172X视频属性的抓取
BV号为1Wv411172X的视频属性抓取结束
开始BV号为1z5411875e视频属性的抓取
BV号为1z5411875e的视频属性抓取结束
开始BV号为1PK4y1v7N6视频属性的抓取
BV号为1PK4y1v7N6的视频属性抓取结束
开始BV号为1rT4y157qW视频属性的抓取
BV号为1rT4y157qW的视频属性抓取结束
开始BV号为1Z541187Wy视频属性的抓取
BV号为1Z541187Wy的视频属性抓取结束
开始BV号为1A64y1F7es视频属性的抓取
BV号为1A64y1F7es的视频属性抓取结束
开始BV号为1iD4y1U7ki视频属性的抓取
BV号为1iD4y1U7ki的视频属性抓取结束
开始BV号为1YK4y1Y7BX视频属性的抓取
BV号为1YK4y1Y7BX的视频属性抓取结束
开始BV号为1y54y1v7Tw视频属性的抓取
BV号为1y54y1v7Tw的视频属性抓取结束
开始BV号为17V411S7yX视频属性的抓取
BV号为17V411S7yX的视

开始BV号为1Wv411C7EV视频属性的抓取
BV号为1Wv411C7EV的视频属性抓取结束
开始BV号为1bf4y1X7S5视频属性的抓取
BV号为1bf4y1X7S5的视频属性抓取结束
开始BV号为1gD4y1o7ff视频属性的抓取
BV号为1gD4y1o7ff的视频属性抓取结束
开始BV号为1FA411J73N视频属性的抓取
BV号为1FA411J73N的视频属性抓取结束
开始BV号为1ni4y1u7q5视频属性的抓取
BV号为1ni4y1u7q5的视频属性抓取结束
开始BV号为1Nh41197nf视频属性的抓取
BV号为1Nh41197nf的视频属性抓取结束
开始BV号为1Jv411C7cW视频属性的抓取
BV号为1Jv411C7cW的视频属性抓取结束
开始BV号为12V41127q5视频属性的抓取
BV号为12V41127q5的视频属性抓取结束
开始BV号为1UZ4y1P7Ab视频属性的抓取
BV号为1UZ4y1P7Ab的视频属性抓取结束
开始BV号为1RD4y1d7ra视频属性的抓取
BV号为1RD4y1d7ra的视频属性抓取结束
开始BV号为1Lt4y1S78g视频属性的抓取
BV号为1Lt4y1S78g的视频属性抓取结束
开始BV号为11K4y1875N视频属性的抓取
BV号为11K4y1875N的视频属性抓取结束
开始BV号为1mV411276u视频属性的抓取
BV号为1mV411276u的视频属性抓取结束
开始BV号为14a4y1j72e视频属性的抓取
BV号为14a4y1j72e的视频属性抓取结束
开始BV号为1VK4y1h7TZ视频属性的抓取
BV号为1VK4y1h7TZ的视频属性抓取结束
开始BV号为1DA411n7Uk视频属性的抓取
BV号为1DA411n7Uk的视频属性抓取结束
开始BV号为1Bk4y1y76X视频属性的抓取
BV号为1Bk4y1y76X的视频属性抓取结束
开始BV号为1GK411K7oW视频属性的抓取
BV号为1GK411K7oW的视频属性抓取结束
开始BV号为1xf4y1X74u视频属性的抓取
BV号为1xf4y1X74u的视频属性抓取结束
开始BV号为1pA411J7GX视频属性的抓取
BV号为1pA411J7GX的视频属性抓取结束
开始BV号为1qp4y1Y7PU视频属性的抓取
BV号为1qp4y1Y7PU的视

In [10]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-10-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [13]:
channel_donghua_duanpian.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [14]:
channel_donghua_duanpian.to_json('channel_donghua_duanpian.json')

In [2]:
pd.read_json('channel_donghua_duanpian.json')

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1wJ411H71u,1月,2020-01-14 10:30:37,肯尼公主kiki,无,9936,5,米 奇 骚 话 屋,动画,短片·手书·配音,活动作品,...,"[搞笑, 动画, 短片·手书·配音, 动漫恶搞, 童年回忆杀, 米奇妙妙屋, 迪士尼, 20...",01:29,583.9万,15.3万,3.2万,9.6万,3.6万,1179,1.9万,是
1MJ411C7ie,1月,2020-01-24 22:54:38,艾叶,无,10.9万,6,互动视频,动画,短片·手书·配音,互动视频,...,"[动画, 短片·手书·配音, 铁心博弈, 2020拜年祭单品]",04:32,463.3万,19.9万,21.5万,12.8万,4.1万,24125,20.6万,否
1RJ411j7kr,1月,2020-01-03 20:06:29,发自内心想吃扇贝,无,2.6万,20,【JOJO】海 澜 之 家,动画,短片·手书·配音,普通视频,...,"[手书, JOJO的奇妙冒险, 动画, 短片·手书·配音, JOJO, DIO]",00:30,297.1万,16.6万,9.6万,13.4万,2.3万,3216,1.6万,否
1qJ411H76x,1月,2020-01-10 17:31:33,帅气上班族吉良,个人认证,17.3万,63,我 不 做 羊 了,动画,短片·手书·配音,活动作品,...,"[喜羊羊, 手书, 搞笑, 动画, 短片·手书·配音, 动漫恶搞, 2019新人嘉年华, 自...",01:18,285.8万,12.7万,4.5万,4.6万,2.7万,1060,4589,否
13J411j7fr,1月,2020-01-02 22:43:49,屏幕另一边的迷云,个人认证,14.2万,1421,大 雄 的 奇 妙 冒 险,动画,短片·手书·配音,普通视频,...,"[哆啦A梦, JOJO的奇妙冒险, 动画, 短片·手书·配音, 童年回忆杀, 黄金之风, J...",02:05,273.5万,8.8万,1.4万,3.5万,1.4万,1155,4268,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1p54y1C7ra,10月,2020-10-30 10:00:56,刘拖地,个人认证,16万,33,“哎呦，他去图书馆学习呢！学霸呀！！”,动画,短片·手书·配音,活动作品,...,"[搞笑配音, 配音, 是在下输了, 动画, 短片·手书·配音, 同人配音, 搞笑剪辑, 十月...",01:30,80.6万,5.3万,8053,8816,5521,503,497,否
15i4y1771y,10月,2020-10-21 12:44:45,功夫阿桩,无,1万,29,三年动画之功夫片：马大师1打20,动画,短片·手书·配音,活动作品,...,"[动画, 短片·手书·配音, 沙雕之主, 大师, 武术, 全能沙雕, 闪电鞭, 五连鞭, 太...",00:48,79.5万,4.4万,1.2万,1.1万,8489,657,743,否
1rV411174D,10月,2020-10-03 20:25:47,空酱w,无,6.8万,37,「翻配日语」瞬间A爆！这样的美总你还不爱？我已经热血起来了,动画,短片·手书·配音,普通视频,...,"[日语, 动画, 短片·手书·配音, 同人配音, 日语配音, 声控, 美羊羊, 动漫配音, 日配]",00:21,78.5万,5.0万,6232,1.8万,2677,550,402,否
1iA41177bZ,10月,2020-10-18 14:51:17,九方朔原,无,2344,7,【描改手书】Jo厨爆肝946张！你从未见过的东方仗助丝滑Say so!,动画,短片·手书·配音,普通视频,...,"[手书, 动画, 短片·手书·配音, 东方仗助, JOJO, 新人, 描改]",00:43,77.4万,9.6万,5.0万,3.2万,2504,812,630,否
